# QUESTION 1

**Q1: Use the dataset “INFO 617 Mental Health QA_LIWC.csv” to answer the following
questions.**

Preparation: convert the numeric variable “Received_Bonus_Yes_No” into a binary variable
“Bonus” that takes a value 1 when “Received_Bonus_Yes_No” is greater than or equal to 1 and 0
otherwise.

Build a CNN (Convolutional Nueral Network) that predicts the binary variable “Bonus” using
texts in the column “Answer_English.” You can use the tutorial we discussed in class as a
reference (INFO_617_Week_11_CNN_for_Text_Classification_(Chris_Tran).ipynb). However,
since “Bonus” has an unbalanced class distribution (i.e., the majority of the answers did not
receive a bonus), you might want to resample the texts before developing the CNN model.

Before the resampling, you should split the dataset into a training set, a validation set, and a test
set. Report your model’s performance on the test set using four metrics, accuracy, precision,
recall, and F1-score. You can experiment with different hyper-parameter settings and pick a well-
performing one to use in your model. However, no formal hyper-parameter tuning is required.

#Importing essential libraries for text preprocessing, visualization, and deep learning model building.
This setup includes tools for data manipulation (pandas, numpy), NLP (nltk), progress tracking (tqdm),
 plotting (matplotlib), and building neural networks (PyTorch).


In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
# For Google Colab integration
import os
from google.colab import drive
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import silhouette_score


from google.colab import drive
drive.mount('/content/drive')

# For data manipulation
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:

# import data as dataframe
file_path = '/content/drive/MyDrive/Colab Notebooks/INFO 617 Mental Health QA_LIWC (1).csv'
df = pd.read_csv(file_path)


# calling head() method
df.head()

,QID,Question_English,Answer_English,Usefulness_vote,Comment_count,Received_Bonus_Yes_No,Segment,WC,Analytic,Clout,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,100788023,Watching the elderly matchmaking program was v...,Hello. Let's start by talking about young peop...,2,0,0,1,359,52.91,83.28,...,0.0,0.0,18.94,6.96,8.36,0.84,0.28,2.23,0.28,0.0
1,100788009,Falling in love with someone else and breaking...,Hello! Sending you a virtual hug first. Thumbs...,2,0,0,1,199,72.83,96.90,...,0.0,0.0,12.06,5.03,5.03,0.00,0.50,0.50,1.01,0.0
2,100788013,I have been suffering from insomnia for half a...,Hello! ☆ I saw the issue you raised about inso...,8,2,1,1,611,37.89,75.35,...,0.0,0.0,15.71,4.75,7.36,0.65,0.16,1.80,0.98,0.0
3,100788013,I have been suffering from insomnia for half a...,Hello! ☆ I saw the issue you raised about inso...,8,2,0,1,611,37.89,75.35,...,0.0,0.0,15.71,4.75,7.36,0.65,0.16,1.80,0.98,0.0
4,100788013,I have been suffering from insomnia for half a...,"Hi, I just want to hug you. I can feel the kin...",6,0,0,1,587,82.45,46.10,...,0.0,0.0,16.18,6.30,4.94,1.19,0.17,0.68,2.90,0.0


In [ ]:
df.describe()

,QID,Usefulness_vote,Comment_count,Received_Bonus_Yes_No,Segment,WC,Analytic,Clout,Authentic,Tone,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
count,8.639700e+04,86397.000000,86397.000000,86397.000000,86397.0,86397.000000,86397.000000,86359.000000,86385.000000,86055.000000,...,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000,86397.000000
mean,1.008195e+08,3.696066,0.188108,0.226779,1.0,361.485677,35.420297,86.609288,37.184117,54.919290,...,0.002742,0.000435,16.942721,5.668207,6.972443,0.473949,0.280939,1.671973,1.875118,0.246987
std,1.494921e+04,2.642346,0.800155,0.431439,0.0,212.424732,20.612153,20.235013,24.168196,31.029913,...,0.057104,0.018663,3.244678,1.499754,1.862775,0.845861,0.518138,1.325822,1.924474,1.114994
min,1.007880e+08,0.000000,0.000000,0.000000,1.0,5.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,3.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.008065e+08,2.000000,0.000000,0.000000,1.0,215.000000,19.120000,84.020000,17.610000,28.140000,...,0.000000,0.000000,14.810000,4.750000,5.800000,0.000000,0.000000,0.750000,0.560000,0.000000
50%,1.008258e+08,3.000000,0.000000,0.000000,1.0,317.000000,32.440000,95.800000,33.740000,57.270000,...,0.000000,0.000000,16.600000,5.530000,6.910000,0.000000,0.000000,1.430000,1.380000,0.000000
75%,1.008321e+08,5.000000,0.000000,0.000000,1.0,469.000000,49.260000,99.000000,53.500000,83.330000,...,0.000000,0.000000,18.640000,6.430000,8.080000,0.640000,0.390000,2.300000,2.620000,0.000000
max,1.008396e+08,31.000000,35.000000,4.000000,1.0,3490.000000,99.000000,99.000000,99.000000,99.000000,...,5.880000,2.880000,68.420000,40.630000,26.670000,20.000000,31.030000,26.320000,38.100000,160.000000


 Convert 'Received_Bonus_Yes_No' into a binary 'Bonus' column:
 1 if bonus was received (value ≥ 1), else 0


In [ ]:
df['Bonus'] = (df['Received_Bonus_Yes_No'] >= 1).astype(int)

Split data into train (80%), validation (10%), and test (10%) with stratification on 'Bonus'
Downsample the majority class in the training set to address class imbalance
 Save all splits for later use


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Bonus"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["Bonus"])

# 4. Downsample ONLY the training set
train_majority = train_df[train_df["Bonus"] == 0]
train_minority = train_df[train_df["Bonus"] == 1]

train_majority_downsampled = resample(train_majority,
                                      replace=False,
                                      n_samples=len(train_minority),
                                      random_state=42)

train_balanced = pd.concat([train_majority_downsampled, train_minority])
train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 5. Save for reference
train_balanced.to_csv("Train_Balanced.csv", index=False)
val_df.to_csv("Val_Original.csv", index=False)
test_df.to_csv("Test_Original.csv", index=False)

print(" Step 1 complete.")
print("→ Train set balanced.")
print("→ Val/Test sets keep original class distribution.")

 Step 1 complete.
→ Train set balanced.
→ Val/Test sets keep original class distribution.


# STEP 2: Preprocess text by tokenizing, building a vocabulary, and encoding input sequences.
 Uses NLTK's word_tokenize to split text into tokens, builds a word2idx mapping,
 and pads/truncates sequences to ensure uniform length across train/val/test sets.


In [ ]:
# STEP 2: Tokenization, Vocab Building, Encoding

import nltk
from nltk.tokenize import word_tokenize
import numpy as np

nltk.download('punkt')

# 1. Load data from step 1
train_df = pd.read_csv("Train_Balanced.csv")
val_df = pd.read_csv("Val_Original.csv")
test_df = pd.read_csv("Test_Original.csv")

# 2. Extract full text and labels
train_texts = train_df["Answer_English"].astype(str).tolist()
val_texts = val_df["Answer_English"].astype(str).tolist()
test_texts = test_df["Answer_English"].astype(str).tolist()

train_labels = train_df["Bonus"].tolist()
val_labels = val_df["Bonus"].tolist()
test_labels = test_df["Bonus"].tolist()

# 3. Tokenization + Vocabulary builder
def tokenize(texts):
    tokenized_texts = []
    word2idx = {"<pad>": 0, "<unk>": 1}
    idx = 2
    max_len = 0

    for sent in texts:
        tokens = word_tokenize(sent.lower())
        tokenized_texts.append(tokens)
        max_len = max(max_len, len(tokens))

        for token in tokens:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

    return tokenized_texts, word2idx, max_len

# 4. Encode and pad the texts
def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokens in tokenized_texts:
        tokens = tokens[:max_len] + ['<pad>'] * max(0, max_len - len(tokens))
        input_ids.append([word2idx.get(token, word2idx["<unk>"]) for token in tokens])
    return np.array(input_ids)

# 5. Tokenize ONLY training set to build vocab
train_tokenized, word2idx, max_len = tokenize(train_texts)

# 6. Tokenize and encode all sets using same vocab + max_len
val_tokenized = [word_tokenize(text.lower()) for text in val_texts]
test_tokenized = [word_tokenize(text.lower()) for text in test_texts]

train_input_ids = encode(train_tokenized, word2idx, max_len)
val_input_ids = encode(val_tokenized, word2idx, max_len)
test_input_ids = encode(test_tokenized, word2idx, max_len)

print(" Tokenization and encoding complete.")
print(f" Vocab size: {len(word2idx)} | Max sequence length: {max_len}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 Tokenization and encoding complete.
 Vocab size: 41555 | Max sequence length: 2803


# STEP 3: Convert encoded sequences and labels into PyTorch tensors.
 Wrap them into TensorDatasets and load them into DataLoaders for efficient batching during training and evaluation.


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert to tensors
train_inputs = torch.tensor(train_input_ids)
val_inputs = torch.tensor(val_input_ids)
test_inputs = torch.tensor(test_input_ids)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

# Create TensorDatasets
train_data = TensorDataset(train_inputs, train_labels)
val_data = TensorDataset(val_inputs, val_labels)
test_data = TensorDataset(test_inputs, test_labels)

batch_size = 50

# DataLoaders
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)
val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=batch_size)
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=batch_size)

print(" DataLoaders ready.")
print(f"Train: {len(train_dataloader)} batches | Val: {len(val_dataloader)} | Test: {len(test_dataloader)}")


 DataLoaders ready.
Train: 613 batches | Val: 173 | Test: 173


# STEP 4: Download GloVe pretrained word vectors (300d) and extract them.
Then, build an embedding matrix aligned with the model's vocabulary (word2idx),
 initializing missing words with random vectors and padding token with zeros.


In [ ]:
import os
import zipfile
import requests
from tqdm import tqdm
import numpy as np

# === STEP 1: Download and extract GloVe === #
EMBED_DIR = "glove"
os.makedirs(EMBED_DIR, exist_ok=True)

GLOVE_URL = "http://nlp.stanford.edu/data/glove.6B.zip"
GLOVE_ZIP_PATH = os.path.join(EMBED_DIR, "glove.6B.zip")

# Download if not already present
if not os.path.exists(GLOVE_ZIP_PATH):
    print("Downloading GloVe vectors...")
    response = requests.get(GLOVE_URL, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    with open(GLOVE_ZIP_PATH, 'wb') as f:
        for data in tqdm(response.iter_content(block_size), total=total_size // block_size, unit='KB', unit_scale=True):
            f.write(data)
    print("Download complete.")

# Extract if not already extracted
glove_txt_path = os.path.join(EMBED_DIR, "glove.6B.300d.txt")
if not os.path.exists(glove_txt_path):
    print("Extracting GloVe vectors...")
    with zipfile.ZipFile(GLOVE_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EMBED_DIR)
    print("Extraction complete.")

# === STEP 2: Build the embedding matrix === #
def load_glove_embeddings(word2idx, embedding_path, embed_dim=300):
    print("Loading GloVe embeddings...")
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), embed_dim))
    embeddings[word2idx.get('<pad>', 0)] = np.zeros(embed_dim)
    found = 0
    with open(embedding_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            values = line.strip().split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            if word in word2idx:
                embeddings[word2idx[word]] = vector
                found += 1
    print(f"Found {found}/{len(word2idx)} words in GloVe.")
    return embeddings


embedding_matrix = load_glove_embeddings(word2idx, glove_txt_path)

print(" Embedding matrix ready.")
print(f"Shape: {embedding_matrix.shape}")

#

842kKB [02:38, 5.31kKB/s]                          


Download complete.
Extracting GloVe vectors...
Extraction complete.
Loading GloVe embeddings...


400000it [00:27, 14480.93it/s]

Found 28099/41555 words in GloVe.
 Embedding matrix ready.
Shape: (41555, 300)


# STEP 5: Define the CNN architecture for text classification.
 This model uses multiple convolutional filters to capture n-gram features,
 followed by max-pooling, dropout for regularization, and a fully connected layer for classification.
 Supports both random and pretrained embeddings (GloVe).


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        super(CNN_NLP, self).__init__()

        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32),
                freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=self.embed_dim,
                padding_idx=0,
                max_norm=5.0)

        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])

        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        x_embed = self.embedding(input_ids).float()
        x_reshaped = x_embed.permute(0, 2, 1)

        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in x_conv_list]

        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list], dim=1)
        logits = self.fc(self.dropout(x_fc))
        return logits


# STEP 6: Initialize the CNN model with given hyperparameters.
 Supports both pretrained and random embeddings. Uses Adadelta optimizer for training.


In [ ]:
import torch.optim as optim

def init_model(pretrained_embedding=None,
               freeze_embedding=False,
               vocab_size=None,
               embed_dim=300,
               filter_sizes=[3, 4, 5],
               num_filters=[100, 100, 100],
               num_classes=2,
               dropout=0.5,
               learning_rate=0.25):

    model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                    freeze_embedding=freeze_embedding,
                    vocab_size=vocab_size,
                    embed_dim=embed_dim,
                    filter_sizes=filter_sizes,
                    num_filters=num_filters,
                    num_classes=num_classes,
                    dropout=dropout)

    model.to(device)

    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, rho=0.95)

    return model, optimizer


# STEP 7: Set the device to GPU if available, otherwise default to CPU


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")


 Using device: cuda


# STEP 8: Define training and evaluation loops.
 The `train()` function runs the training process across multiple epochs, logging loss and accuracy.
 The `evaluate()` function computes average loss and accuracy on validation/test datasets without updating model weights.


In [ ]:
import time

# Loss function
loss_fn = nn.CrossEntropyLoss()

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    best_val_acc = 0

    print(" Training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Time':^9}")
    print("-" * 60)

    for epoch in range(epochs):
        t0 = time.time()
        total_loss = 0
        model.train()

        for batch in train_dataloader:
            b_input_ids, b_labels = [t.to(device) for t in batch]
            model.zero_grad()

            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_dataloader)

        # Validate
        if val_dataloader:
            val_loss, val_acc = evaluate(model, val_dataloader)
            if val_acc > best_val_acc:
                best_val_acc = val_acc

            elapsed = time.time() - t0
            print(f"{epoch + 1:^7} | {avg_train_loss:^12.4f} | {val_loss:^10.4f} | {val_acc:^9.2f} | {elapsed:^9.2f}")

    print("\n Training complete! Best val accuracy:", round(best_val_acc, 2), "%")

# Evaluation loop (validation or test)
def evaluate(model, dataloader):
    model.eval()
    val_loss = []
    val_acc = []

    with torch.no_grad():
        for batch in dataloader:
            b_input_ids, b_labels = [t.to(device) for t in batch]
            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            val_loss.append(loss.item())

            preds = torch.argmax(logits, dim=1)
            acc = (preds == b_labels).cpu().numpy().mean() * 100
            val_acc.append(acc)

    return np.mean(val_loss), np.mean(val_acc)


# STEP 9: Train three CNN model variants:
 1. CNN-rand: Uses randomly initialized embeddings
 2. CNN-static: Uses pretrained GloVe embeddings (frozen)
 3. CNN-non-static: Uses pretrained GloVe embeddings (fine-tuned during training)


In [ ]:
# CNN-rand: randomly initialized embeddings
print("\n Training CNN-rand (random embeddings)")
cnn_rand, opt_rand = init_model(vocab_size=len(word2idx))
train(cnn_rand, opt_rand, train_dataloader, val_dataloader, epochs=10)

# CNN-static: pretrained fastText, frozen
print("\n Training CNN-static (pretrained, frozen)")
cnn_static, opt_static = init_model(pretrained_embedding=embedding_matrix,
                                    freeze_embedding=True)
train(cnn_static, opt_static, train_dataloader, val_dataloader, epochs=10)

# CNN-non-static: pretrained fastText, fine-tuned
print("\n Training CNN-non-static (pretrained, trainable)")
cnn_non_static, opt_non_static = init_model(pretrained_embedding=embedding_matrix,
                                            freeze_embedding=False)
train(cnn_non_static, opt_non_static, train_dataloader, val_dataloader, epochs=10)


 Training CNN-rand (random embeddings)
 Training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |   Time   
------------------------------------------------------------
   1    |    0.6212    |   0.5995   |   67.86   |   88.61  
   2    |    0.5529    |   0.5774   |   69.67   |   91.56  
   3    |    0.5184    |   0.6334   |   65.91   |   93.72  
   4    |    0.4930    |   0.4470   |   79.00   |   94.60  
   5    |    0.4673    |   0.4865   |   76.03   |   94.98  
   6    |    0.4428    |   0.4714   |   77.55   |   95.18  
   7    |    0.4181    |   0.5986   |   70.81   |   95.39  
   8    |    0.3929    |   0.4724   |   77.84   |   95.34  
   9    |    0.3632    |   0.6011   |   71.98   |   95.46  
  10    |    0.3408    |   0.4014   |   81.63   |   95.40  

 Training complete! Best val accuracy: 81.63 %

 Training CNN-static (pretrained, frozen)
 Training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |   Time   
------------------------------------------------------------

# STEP 10: Training Results Summary
 CNN-rand (random embeddings) achieved best val accuracy: **81.63%**# CNN-static (pretrained, frozen) achieved slightly higher accuracy: **81.86%**
 CNN-non-static (pretrained, trainable) peaked at: **75.91%**

# Takeaway:
 - Pretrained embeddings (GloVe) provided a solid starting point.
 - Freezing the embeddings (static) performed slightly better than fine-tuning them (non-static),
   likely due to overfitting or sensitivity to training data.
 - Random initialization (CNN-rand) performed competitively, showing the strength of the architecture alone.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_on_test(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            b_input_ids, b_labels = [t.to(device) for t in batch]
            logits = model(b_input_ids)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(b_labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return acc, prec, rec, f1


# STEP 11: Evaluate all trained models on the test set using accuracy, precision, recall, and F1-score.
 Compare how each variant (random, static, non-static) generalizes to unseen data.


In [ ]:
models = [
    ("CNN-rand", cnn_rand),
    ("CNN-static", cnn_static),
    ("CNN-non-static", cnn_non_static)
]

for name, model in models:
    acc, prec, rec, f1 = evaluate_on_test(model, test_dataloader)
    print(f"{name} Test Set Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("-" * 40)


CNN-rand Test Set Evaluation:
Accuracy:  0.8102
Precision: 0.5612
Recall:    0.6580
F1-score:  0.6058
----------------------------------------
CNN-static Test Set Evaluation:
Accuracy:  0.7657
Precision: 0.4824
Recall:    0.7802
F1-score:  0.5962
----------------------------------------
CNN-non-static Test Set Evaluation:
Accuracy:  0.7257
Precision: 0.4395
Recall:    0.8632
F1-score:  0.5825
----------------------------------------


# STEP 12: Test Set Performance Summary

#  CNN-rand
 - Best overall **accuracy** (81.02%) and **F1-score** (0.6058)
 - Balanced performance, strong generalization despite no pretrained knowledge

#  CNN-static
 - Highest **recall** among the pretrained models (78.02%)
- Performs well at catching positive cases, but lower precision

#  CNN-non-static
 - **Highest recall** (86.32%), but with a **lower precision** (43.95%)
 - Tends to over-predict positives, resulting in lower overall accuracy

# Takeaway:
 - Random embeddings surprisingly performed best overall.
 - Fine-tuning pretrained embeddings (non-static) increased recall at the cost of precision.
 - Freezing pretrained embeddings (static) gave balanced but slightly underperforming results.
